In [1]:
import pandas as pd
import os
import yfinance as yf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from itertools import chain, combinations
import statsmodels.formula.api as smf
import scipy.stats as scipystats
import statsmodels.api as sm
import statsmodels.stats.stattools as stools
import statsmodels.stats as stats
from statsmodels.graphics.regressionplots import *
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import math
import time

In [ ]:
stocks = pd.read_csv('stocks_5_groups.csv')

stocks['day']=pd.to_datetime(stocks.trade_date)
pd.set_option('display.max_columns', None)

hh_h=stocks[stocks.group==5]
etfs = pd.read_csv('etf_data_5_groups.csv')
etfs['day']=pd.to_datetime(etfs.etf_trade_date)
etfs=etfs.loc[etfs.day > '2020-07-01']
etfs.head()

In [ ]:
spy= etfs[etfs.etf_symbol=='SPY']
spy= spy[['day', 'etf_close_price', 'etf_close_size',
       'etf_close_time', 'etf_intraday_max_price', 'etf_intraday_median_price',
       'etf_intraday_min_price', 'etf_intraday_std_price',
       'etf_intraday_volume', 'etf_open_price', 'etf_open_size',
       'etf_open_time', 'etf_post_max_price', 'etf_post_median_price',
       'etf_post_min_price', 'etf_post_std_price', 'etf_post_volume',
       'etf_pre_max_price', 'etf_pre_median_price', 'etf_pre_min_price',
       'etf_pre_std_price', 'etf_pre_volume', 'etf_trade_date',
       ]]
spy.columns= ['day', 'spy_close_price', 'spy_close_size', 'spy_close_time',
       'spy_intraday_max_price', 'spy_intraday_median_price',
       'spy_intraday_min_price', 'spy_intraday_std_price',
       'spy_intraday_volume', 'spy_open_price', 'spy_open_size',
       'spy_open_time', 'spy_post_max_price', 'spy_post_median_price',
       'spy_post_min_price', 'spy_post_std_price', 'spy_post_volume',
       'spy_pre_max_price', 'spy_pre_median_price', 'spy_pre_min_price',
       'spy_pre_std_price', 'spy_pre_volume', 'spy_trade_date']

spy['spy_prev_close_to_open_ret'] = spy['spy_open_price']/spy['spy_close_price'].shift(1) -1
spy['spy_prev_close_pre_med_ret'] = spy['spy_pre_median_price']/spy['spy_close_price'].shift(1)-1 

spy['spy_pre_combined_volume']= spy['spy_post_volume'].shift(1) +  spy['spy_pre_volume']
spy['spy_pre_combined_adv']=  spy['spy_pre_combined_volume'].rolling(20).quantile(0.5)

spy['spy_intraday_adv']=  spy['spy_intraday_volume'].rolling(20).quantile(0.5)
spy['intraday_relative_volume_lag1']= spy['spy_intraday_volume'].shift(1)/ spy['spy_intraday_adv']
spy['row'] = np.arange(len(spy))
spy = spy.set_index('row')
spy=spy[(spy.day > '2020-07-01') & (spy.day < '2021-01-21')]

In [ ]:
spy['spy_prev_close_to_open_ret'] = spy['spy_open_price']/spy['spy_close_price'].shift(1) -1
spy['spy_prev_close_pre_med_ret'] = spy['spy_pre_median_price']/spy['spy_close_price'].shift(1)-1 

spy['rel_lag_intraday_volume']= 
spy['spy_pre_combined_volume']= spy['spy_post_volume'].shift(1) +  spy['spy_pre_volume']
spy['spy_pre_combined_adv']=  spy['spy_pre_combined_volume'].rolling(20).quantile(0.5)

spy['spy_intraday_adv']=  spy['spy_intraday_volume'].rolling(20).quantile(0.5)
spy['rel_lag_intraday_relative_volume']= spy['spy_intraday_volume']/ spy['spy_intraday_adv']
spy_days = spy[['day']]
spy_days.columns=['day']

In [ ]:
splits = pd.read_csv('split-5-groups.csv')
#splits['symbol']=splits.split_ticker
splits['day']=pd.to_datetime(splits.split_exDate)
splits['symbol']= splits.split_ticker
splits=splits.loc[(splits.day > '2020-07-01') & (splits.day < '2021-01-21')]
splits

In [ ]:
divs = pd.read_csv('div-for-5groups.csv')
divs['symbol']=divs.dividend_ticker
divs['day']=pd.to_datetime(divs.dividend_exDate)
divs=divs.loc[(divs.day > '2020-07-01') & (divs.day < '2021-01-21')]
divs.head()

In [ ]:

orders= pd.read_csv('alltradedata-20210120.csv')
orders['created_date'] = pd.to_datetime(orders['created_date'])
orders['day'] = orders.created_date.dt.date
orders['time']= orders.created_date.dt.time
orders['hour']= orders.created_date.dt.hour
orders['minute']= orders.created_date.dt.minute
orders['weekday'] = pd.to_datetime(orders['created_date']).dt.dayofweek
orders['symbol']= orders.original_market_symbol
orders['share_price']= orders.share_price
orders['share_filled']= orders['share_filled'].replace(np.nan, 0)
orders['share_price']= orders['share_price'].replace(np.nan, 0)
orders['day']=pd.to_datetime(orders.day)
orders= orders[orders.day > '2020-07-01']
symbol= 'TSLA'
sym_i_divs= divs[divs.symbol==symbol]
sym_i_divs= sym_i_divs[['day','dividend_amount']]
sym_i_splits= splits[splits.symbol==symbol]
sym_i_splits= sym_i_splits[['day','split_forfactor']]
sym_actions =pd.merge(spy_days , sym_i_divs ,  on='day', how='outer')
sym_actions =pd.merge(sym_actions, sym_i_splits ,  on='day', how='outer')
sym_actions['dividend_amount']= sym_actions['dividend_amount'].replace(np.nan, 0)
sym_actions['split_forfactor']= sym_actions['split_forfactor'].replace(np.nan, 1)
sym_actions

In [ ]:
hh_h = pd.read_csv('data_only_group_5.csv')
hh_h['day']=pd.to_datetime(hh_h.trade_date)
hh_h= hh_h[(hh_h.day > '2020-07-01') & (hh_h.day < '2021-01-21')]

In [ ]:
orders_i=orders[orders.symbol==symbol]
orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
orders_i=orders[orders.symbol==symbol]

orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
orders_i['nshares'] = orders_i.side * orders_i.share_filled

orders_i= orders_i.set_index('created_date')
order_i_post=orders_i.between_time('16:00', '23:59')
order_i_pre=orders_i.between_time('00:00', '09:30')
order_i_intraday =orders_i.between_time('09:31', '16:00')

orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
orders_i['nshares'] = orders_i.side * orders_i.share_filled

order_i_days = orders_i[['day']]
order_i_days = order_i_days.day.unique()
order_i_days= pd.DataFrame(order_i_days)
order_i_days.columns=['day']

In [ ]:
sym_i = hh_h[hh_h.symbol==symbol]
sym_i= pd.merge(sym_i, sym_actions, on='day')
sym_i = sym_i.sort_values(by=['day'], ascending=True)

sym_i['prev_close_price'] = sym_i['close_price'].shift(1)
sym_i['prev_post_volume'] = sym_i['post_volume'].shift(1)

sym_i['pre_combined_volume']= sym_i['prev_post_volume'] +  sym_i['pre_volume']

sym_i['pre_combined_adv']=  sym_i['pre_combined_volume'].rolling(20).quantile(0.5)
sym_i['pre_adv']= sym_i['pre_volume'].rolling(20).quantile(0.5)
sym_i['post_adv']= sym_i['post_volume'].rolling(20).quantile(0.5)
sym_i['intraday_adv']= sym_i['intraday_volume'].rolling(20).quantile(0.5)
sym_i['prev_close_to_open_ret'] = (sym_i.open_price*sym_i.split_forfactor-sym_i.dividend_amount)/sym_i.prev_close_price -1
sym_i['prev_close_to_pre_med_ret'] = (sym_i.pre_median_price*sym_i.split_forfactor-sym_i.dividend_amount)/sym_i.prev_close_price -1

In [ ]:

orders_i=orders[orders.symbol==symbol]
orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
orders_i=orders[orders.symbol==symbol]

orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
orders_i['nshares'] = orders_i.side * orders_i.share_filled

orders_i= orders_i.set_index('created_date')
order_i_post=orders_i.between_time('16:00', '23:59')
order_i_pre=orders_i.between_time('00:00', '09:30')
order_i_intraday =orders_i.between_time('09:31', '16:00')

orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
orders_i['nshares'] = orders_i.side * orders_i.share_filled

order_i_days = orders_i[['day']]
order_i_days = order_i_days.day.unique()
order_i_days= pd.DataFrame(order_i_days)
order_i_days.columns=['day']

pre_orders= pd.DataFrame( order_i_pre.groupby(['day'])['nshares'].sum() )
pre_orders['day'] = pre_orders.index
pre_orders['row'] = np.arange(len(pre_orders))
pre_orders = pre_orders.set_index('row')
pre_orders_i =pd.merge(order_i_days , pre_orders,  on='day', how='outer')
pre_orders_i.columns=['day', 'pre_nshares']

post_orders= pd.DataFrame( order_i_post.groupby(['day'])['nshares'].sum() )
post_orders['day'] = post_orders.index
post_orders['row'] = np.arange(len(post_orders))
post_orders = post_orders.set_index('row')
post_orders_i =pd.merge(order_i_days , post_orders,  on='day', how='outer')
post_orders_i.columns=['day', 'post_nshares']

intraday_orders= pd.DataFrame( order_i_intraday.groupby(['day'])['nshares'].sum() )
intraday_orders['day'] = intraday_orders.index
intraday_orders['row'] = np.arange(len(intraday_orders))
intraday_orders = intraday_orders.set_index('row')
intraday_orders_i =pd.merge(order_i_days , intraday_orders,  on='day', how='outer')
intraday_orders_i.columns=['day', 'intraday_nshares']

order_smry_i =pd.merge(pre_orders_i, intraday_orders_i, on='day', how='outer')
order_smry_i =pd.merge(order_smry_i, post_orders_i, on='day', how='outer')

order_smry_i = order_smry_i.sort_values(by=['day'], ascending=True)

order_smry_i['weekday'] = pd.to_datetime(order_smry_i['day']).dt.dayofweek
order_smry_i['pre_nshares']= order_smry_i['pre_nshares'].replace(np.nan, 0)
order_smry_i['post_nshares']= order_smry_i['post_nshares'].replace(np.nan, 0)
order_smry_i['intraday_nshares']= order_smry_i['intraday_nshares'].replace(np.nan, 0)

order_smry_i['tot_nshares']= order_smry_i['pre_nshares'] + order_smry_i['intraday_nshares']+ order_smry_i['post_nshares']
order_smry_i['tot_nshares_lag1']= order_smry_i['tot_nshares'].shift(1)
order_smry_i['tot_nshares_lag2']= order_smry_i['tot_nshares'].shift(2)
order_smry_i['post_nshares_lag3'] = order_smry_i['post_nshares'].shift(3)
order_smry_i['pre_Monday_open_shares']= order_smry_i['tot_nshares_lag1'] + order_smry_i['tot_nshares_lag2']+order_smry_i['post_nshares_lag3']
order_smry_i['pre_open_shares']= order_smry_i['pre_nshares'] + order_smry_i['post_nshares'].shift(1)

order_smry_i['pre_open_tot_nshares']= np.where(order_smry_i.weekday ==0 , order_smry_i['pre_Monday_open_shares'], order_smry_i['pre_open_shares'])

In [ ]:

syms=hh_h['symbol'].unique()
In [218]:
i = 0
symbol = syms[0]
symbol
Out[218]:
'TSLA'
In [242]:
import time 
In [258]:
mkt_orders = pd.DataFrame()
In [259]:
i0 = 0
for i in range(i0,len(syms)):
    start=time.perf_counter()
    symbol = syms[i]
    print (i, symbol)
    # get the symbol's action data
    sym_i_divs= divs[divs.symbol==symbol]
    sym_i_divs= sym_i_divs[['day','dividend_amount']]
    sym_i_splits= splits[splits.symbol==symbol]
    sym_i_splits= sym_i_splits[['day','split_forfactor']]
    sym_actions =pd.merge(spy_days , sym_i_divs ,  on='day', how='outer')
    sym_actions =pd.merge(sym_actions, sym_i_splits ,  on='day', how='outer')
    sym_actions['dividend_amount']= sym_actions['dividend_amount'].replace(np.nan, 0)
    sym_actions['split_forfactor']= sym_actions['split_forfactor'].replace(np.nan, 1)
    # get symbols market data
   
    sym_i = hh_h[hh_h.symbol==symbol]
    sym_i= pd.merge(sym_i, sym_actions , on='day')
    sym_i = sym_i.sort_values(by=['day'], ascending=True)
    
    sym_i['prev_close_price'] = sym_i['close_price'].shift(1)
    sym_i['prev_post_volume'] = sym_i['post_volume'].shift(1)

    sym_i['pre_combined_volume']= sym_i['prev_post_volume'] +  sym_i['pre_volume']
    sym_i['pre_combined_adv']=  sym_i['pre_combined_volume'].rolling(20).quantile(0.5)
    sym_i['pre_adv']= sym_i['pre_volume'].rolling(20).quantile(0.5)
    sym_i['post_adv']= sym_i['post_volume'].rolling(20).quantile(0.5)
    sym_i['intraday_adv']= sym_i['intraday_volume'].rolling(20).quantile(0.5)
    sym_i['lag_intraday_relative_volume']= sym_i['intraday_volume'].shift(1)/ sym_i['intraday_adv']
    
    sym_i['prev_close_to_open_ret'] = (sym_i.open_price*sym_i.split_forfactor-sym_i.dividend_amount)/sym_i.prev_close_price -1
    sym_i['prev_close_to_pre_med_ret'] = (sym_i.pre_median_price*sym_i.split_forfactor-sym_i.dividend_amount)/sym_i.prev_close_price -1
    
    orders_i=orders[orders.symbol==symbol]
    orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
    orders_i=orders[orders.symbol==symbol]

    orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
    orders_i['nshares'] = orders_i.side * orders_i.share_filled

    orders_i= orders_i.set_index('created_date')
    order_i_post=orders_i.between_time('16:00', '23:59')
    order_i_pre=orders_i.between_time('00:00', '09:30')
    order_i_intraday =orders_i.between_time('09:31', '16:00')

    orders_i['side']=np.where(orders_i.operation=='BUY' , 1, -1)
    orders_i['nshares'] = orders_i.side * orders_i.share_filled

    order_i_days = orders_i[['day']]
    order_i_days = order_i_days.day.unique()
    order_i_days= pd.DataFrame(order_i_days)
    order_i_days.columns=['day']

    pre_orders= pd.DataFrame( order_i_pre.groupby(['day'])['nshares'].sum() )
    pre_orders['day'] = pre_orders.index
    pre_orders['row'] = np.arange(len(pre_orders))
    pre_orders = pre_orders.set_index('row')
    pre_orders_i =pd.merge(order_i_days , pre_orders,  on='day', how='outer')
    pre_orders_i.columns=['day', 'pre_nshares']

    post_orders= pd.DataFrame( order_i_post.groupby(['day'])['nshares'].sum() )
    post_orders['day'] = post_orders.index
    post_orders['row'] = np.arange(len(post_orders))
    post_orders = post_orders.set_index('row')
    post_orders_i =pd.merge(order_i_days , post_orders,  on='day', how='outer')
    post_orders_i.columns=['day', 'post_nshares']

    intraday_orders= pd.DataFrame( order_i_intraday.groupby(['day'])['nshares'].sum() )
    intraday_orders['day'] = intraday_orders.index
    intraday_orders['row'] = np.arange(len(intraday_orders))
    intraday_orders = intraday_orders.set_index('row')
    intraday_orders_i =pd.merge(order_i_days , intraday_orders,  on='day', how='outer')
    intraday_orders_i.columns=['day', 'intraday_nshares']

    order_smry_i =pd.merge(pre_orders_i, intraday_orders_i, on='day', how='outer')
    order_smry_i =pd.merge(order_smry_i, post_orders_i, on='day', how='outer')

    order_smry_i = order_smry_i.sort_values(by=['day'], ascending=True)

    order_smry_i['weekday'] = pd.to_datetime(order_smry_i['day']).dt.dayofweek
    order_smry_i['pre_nshares']= order_smry_i['pre_nshares'].replace(np.nan, 0)
    order_smry_i['post_nshares']= order_smry_i['post_nshares'].replace(np.nan, 0)
    order_smry_i['intraday_nshares']= order_smry_i['intraday_nshares'].replace(np.nan, 0)

    order_smry_i['tot_nshares']= order_smry_i['pre_nshares'] + order_smry_i['intraday_nshares']+ order_smry_i['post_nshares']
    order_smry_i['tot_nshares_lag1']= order_smry_i['tot_nshares'].shift(1)
    order_smry_i['tot_nshares_lag2']= order_smry_i['tot_nshares'].shift(2)
    order_smry_i['post_nshares_lag3'] = order_smry_i['post_nshares'].shift(3)
    order_smry_i['pre_Monday_open_shares']= order_smry_i['tot_nshares_lag1'] + order_smry_i['tot_nshares_lag2']+order_smry_i['post_nshares_lag3']
    order_smry_i['pre_open_shares']= order_smry_i['pre_nshares'] + order_smry_i['post_nshares'].shift(1)

    order_smry_i['pre_open_tot_nshares']= np.where(order_smry_i.weekday ==0 , order_smry_i['pre_Monday_open_shares'], order_smry_i['pre_open_shares'])
    
    sym_i = pd.merge(sym_i  ,spy, on = 'day')
    market_orders_i= pd.merge(sym_i , order_smry_i , on = 'day')
    mkt_orders = mkt_orders.append(market_orders_i)  
         
    end=time.perf_counter() # measures the time the iteration ends
    print(symbol, ' ' ,f'processing time {end-start:0.4f} seconds' )

In [ ]:
mkt_orders.to_csv('market_data_andOrders_g5.csv')


In [ ]:
def read_data_files(start, end, stocks_file, etfs_file, div_file, splits_file, orders_file):
    